In [145]:
import argparse
import datetime
import time
import cv2
import numpy as np
import os

import yaml

In [146]:
#filePath = "/home/joe/Videos/Webcam/2022-04-20-210347.webm"
filePath = "/home/joe/Videos/foos_rectified2.avi"
camera = cv2.VideoCapture(filePath)

In [147]:
pinkLower = (140, 10, 225)
pinkUpper = (180, 118, 255)

redLower = (17, 15, 100)
redUpper = (50, 56, 200)

upper = pinkUpper
lower = redUpper

In [148]:
(grabbed, frame) = camera.read()
height, width = frame.shape[:2]
w = width
h = height



In [149]:
calibrationFile = open('calibrations/calibration2.yaml', 'r')
calibrationData = yaml.safe_load(calibrationFile)

camera_matrix = calibrationData['camera_matrix']
numColumns = camera_matrix['cols']
numRows =  camera_matrix['rows']
cameraMatrix = np.array(camera_matrix['data'])
cameraMatrix = np.reshape(cameraMatrix, (numRows, numColumns))

distortion_coefficients = calibrationData['distortion_coefficients']
numColumns = distortion_coefficients['cols']
numRows =  distortion_coefficients['rows']
distortionCoefficients = np.array(distortion_coefficients['data'])
#distortionCoefficients = np.reshape(distortionCoefficients, (numRows, numColumns))



In [150]:
#newcameramtx, roi=cv2.getOptimalNewCameraMatrix(cameraMatrix,distortionCoefficients,(w,h),1,(w,h))
#frame = cv2.undistort(frame, cameraMatrix, distortionCoefficients, None, newcameramtx) 


In [151]:
diagnosticFrame = frame.copy()

In [152]:
markerLength = 0.02 #2cm, 0.02m
distCoeffs = distortionCoefficients
(rvecs, tvecs, points) = cv2.aruco.estimatePoseSingleMarkers(arucoCorners, markerLength, cameraMatrix, distCoeffs)

In [153]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
arucoParams = cv2.aruco.DetectorParameters_create()


ballDetectionCorners = [(0,0), (0, height), (height,width), (0,width)]

In [154]:
(arucoCorners, ids, rejected) = cv2.aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

In [155]:
if(len(corners) > 0):
		ids = ids.flatten()

		for(markerCorner, markerID) in zip(arucoCorners, ids):
			corners = markerCorner.reshape((4,2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners


			# convert each of the (x, y)-coordinate pairs to integers
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))


			cv2.line(diagnosticFrame, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(diagnosticFrame, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(diagnosticFrame, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(diagnosticFrame, bottomLeft, topLeft, (0, 255, 0), 2)

			if(markerID >= 0 and markerID <= 3):
				newCorner = (0,0)
				if(markerID == 0):
					newCorner = topRight
				elif (markerID == 1):
					newCorner = bottomRight
				elif (markerID == 2):
					newCorner = bottomLeft
				elif (markerID == 3):
					newCorner = topLeft
				ballDetectionCorners[markerID] = newCorner

In [156]:
#draw bounding box
boundingBoxPoints = np.array(ballDetectionCorners)
boundingBoxPoints.reshape((-1,1,2))
cv2.polylines(diagnosticFrame, [boundingBoxPoints], True, (255, 0, 0), 2)

array([[[ 32,  18,   0],
        [ 33,  19,   0],
        [ 36,  22,   3],
        ...,
        [139, 139, 139],
        [132, 132, 132],
        [121, 121, 121]],

       [[ 32,  18,   0],
        [ 33,  19,   0],
        [ 36,  22,   3],
        ...,
        [144, 144, 144],
        [135, 135, 135],
        [125, 125, 125]],

       [[ 32,  18,   0],
        [ 33,  19,   0],
        [ 36,  22,   3],
        ...,
        [153, 153, 153],
        [142, 142, 142],
        [132, 132, 132]],

       ...,

       [[132, 102,  77],
        [126,  96,  71],
        [121,  91,  66],
        ...,
        [132, 115, 112],
        [127, 110, 107],
        [131, 114, 111]],

       [[133, 103,  78],
        [127,  97,  72],
        [122,  92,  67],
        ...,
        [133, 116, 113],
        [126, 109, 106],
        [132, 115, 112]],

       [[134, 104,  79],
        [127,  97,  72],
        [123,  93,  68],
        ...,
        [133, 116, 113],
        [126, 109, 106],
        [132, 115, 112]]

In [157]:
print(rvecs)

[[[-0.10706639 -2.24549753  0.13901433]]

 [[-0.07102491 -2.95078764  0.70873163]]

 [[ 0.00458704  2.99962231  0.35205048]]

 [[-0.14573697  3.12850815  0.33595288]]]


In [158]:
print(tvecs)

[[[ 0.13088898  0.12455239  0.43499299]]

 [[-0.21993868  0.11692066  0.40872502]]

 [[ 0.15010787 -0.14035634  0.51879189]]

 [[-0.23541319 -0.14070004  0.47387698]]]


In [159]:
print(points)

[[[-0.01  0.01  0.  ]]

 [[ 0.01  0.01  0.  ]]

 [[ 0.01 -0.01  0.  ]]

 [[-0.01 -0.01  0.  ]]]


In [160]:
for rvec, tvec in zip(rvecs, tvecs):
    diagnosticFrame = cv2.drawFrameAxes(diagnosticFrame, cameraMatrix, distCoeffs, rvec, tvec, 0.05)

In [161]:
cv2.imwrite("test_img7.png", diagnosticFrame)

True

In [144]:
camera.release()

In [127]:
prev_t_vecs = tvecs
prev_rvecs = rvecs